In [3]:
import torch
import numpy as np


In [4]:
def zero_pad_weights(weights):
    i = 0
    res = np.zeros(len(weights) + 7*len(weights)//9).flatten()

    counter = 0
    for w in weights:
        # print(w)
        if counter == 9:
            i += 7
            counter = 0
        res[i] = w
        counter += 1
        i += 1

    return res

def zero_pad_layers(layers):
    i = 0
    res = np.zeros(len(layers) + 15*len(layers)).flatten()
    print(len(layers)+4096*15)
    counter = 0
    for w in layers:
        res[i*16] = w
        i += 1
    return res


In [5]:

CHECKPOINT_PATH = "latest_checkpoint.pth"

q = 8
scalse_factor = 2 ** q

try:
    checkpoint = torch.load(CHECKPOINT_PATH, map_location="cpu")
    gen_state = checkpoint["gen"] if "gen" in checkpoint else checkpoint
    # print("=> File berhasil dimuat.")
except FileNotFoundError:
    print("=> ERROR: File tidak ditemukan.")
    exit()

def get_scaled_weight(tensor_weight):
    w = tensor_weight
    # Shape Conv: (Out, In, K, K)
    # Fan In = In * K * K
    fan_in = w.shape[1] * w.shape[2] * w.shape[3]
    scale = np.sqrt(2 / fan_in)
    return w * scale

# Wadah untuk weight NumPy
weights_numpy = {}

# --- 1. EKSTRAKSI BLOK AWAL (4x4) ---
# print("\n--- Memuat Blok Awal (4x4) ---")

# Ambil Konstanta
weights_numpy['const_input'] = ((gen_state["starting_constant"].numpy())* scalse_factor).astype(np.int16)
flat_weight = weights_numpy['const_input'].flatten()
flat_weight = zero_pad_layers(flat_weight)
flat_weight.tofile('weights_const_input.bin')
# print(f"[OK] Constant Input: {weights_numpy['const_input'].shape}")

# Ambil Konvolusi (Perhatikan key-nya!)
weights_numpy['conv_4x4'] = (get_scaled_weight(gen_state["initial_conv.weight"].numpy())* scalse_factor).astype(np.int16)
weights_numpy['bias_4x4'] = (gen_state["initial_conv.bias"].numpy() * scalse_factor).astype(np.int16)


flat_weight = zero_pad_weights(weights_numpy['conv_4x4'].flatten())
flat_weight.tofile('weights_conv_4x4.bin')

flat_weight = weights_numpy['bias_4x4'].flatten()
flat_weight.tofile('weights_bias_4x4.bin')
# print(f"[OK] Initial Conv: {weights_numpy['conv_4x4'].shape}")

# Ambil Noise
weights_numpy['noise1_4x4'] = (gen_state["initial_noise1.weight"].numpy() * scalse_factor).astype(np.int16)
weights_numpy['noise2_4x4'] = (gen_state["initial_noise2.weight"].numpy() * scalse_factor).astype(np.int16)

flat_weight = weights_numpy['noise1_4x4'].flatten()
flat_weight.tofile('weights_noise1_4x4.bin')
flat_weight = weights_numpy['noise2_4x4'].flatten()
flat_weight.tofile('weights_noise2_4x4.bin')

# --- 2. EKSTRAKSI BLOK PROGRESIF (8x8 ke atas) ---
# Gunakan logika loop seperti sebelumnya, tapi mulai dari index 0
num_blocks = 0
while f"prog_blocks.{num_blocks}.conv1.conv.weight" in gen_state:
    num_blocks += 1

# print(f"\n--- Memuat {num_blocks} Blok Progresif ---")


for i in range(num_blocks):
    res = 4 * (2 ** (i + 1)) # 8, 16, 32...
    
    # Conv 1
    k1 = (gen_state[f"prog_blocks.{i}.conv1.conv.weight"].numpy())
    weights_numpy[f'conv1_{res}'] = (get_scaled_weight(k1)* scalse_factor).astype(np.int16)
    fixed_weight = (weights_numpy[f'conv1_{res}'] )
    flat_weight = fixed_weight.flatten()
    flat_weight = zero_pad_weights(flat_weight)
    flat_weight.tofile(f'weights_conv1_{res}.bin')
    
    # Conv 2
    k2 = gen_state[f"prog_blocks.{i}.conv2.conv.weight"].numpy()
    weights_numpy[f'conv2_{res}'] = (get_scaled_weight(k2)* scalse_factor).astype(np.int16)

    fixed_weight = (weights_numpy[f'conv2_{res}'] )
    flat_weight = fixed_weight.flatten()
    flat_weight = zero_pad_weights(flat_weight)
    flat_weight.tofile(f'weights_conv2_{res}.bin')
    
    # Noise
    n1 = gen_state[f"prog_blocks.{i}.inject_noise1.weight"].numpy()
    n2 = gen_state[f"prog_blocks.{i}.inject_noise2.weight"].numpy()
    weights_numpy[f'noise1_{res}'] = (n1 * scalse_factor).astype(np.int16)
    weights_numpy[f'noise2_{res}'] = (n2 * scalse_factor).astype(np.int16)

    fixed_weight = (weights_numpy[f'noise1_{res}'])
    flat_weight = fixed_weight.flatten()
    flat_weight.tofile(f'weights_noise1_{res}.bin')

    fixed_weight = (weights_numpy[f'noise2_{res}'])
    flat_weight = fixed_weight.flatten()
    flat_weight.tofile(f'weights_noise2_{res}.bin')

    # bias
    b1 = gen_state[f"prog_blocks.{i}.conv1.bias"].numpy()
    b2 = gen_state[f"prog_blocks.{i}.conv2.bias"].numpy()
    weights_numpy[f'bias1_{res}'] = (b1 * scalse_factor).astype(np.int16)
    weights_numpy[f'bias2_{res}'] = (b2 * scalse_factor).astype(np.int16)

    fixed_weight = (weights_numpy[f'bias1_{res}'])
    flat_weight = fixed_weight.flatten()
    flat_weight.tofile(f'weights_bias1_{res}.bin')

    fixed_weight = (weights_numpy[f'bias2_{res}'])
    flat_weight = fixed_weight.flatten()
    flat_weight.tofile(f'weights_bias2_{res}.bin')
    
    print(f"Block {res}x{res} loaded. Conv1: {k1.shape}, Conv2: {k2.shape}, {n1.dtype}")



65536
Block 8x8 loaded. Conv1: (256, 256, 3, 3), Conv2: (256, 256, 3, 3), float32
Block 16x16 loaded. Conv1: (256, 256, 3, 3), Conv2: (256, 256, 3, 3), float32
Block 32x32 loaded. Conv1: (256, 256, 3, 3), Conv2: (256, 256, 3, 3), float32
Block 64x64 loaded. Conv1: (128, 256, 3, 3), Conv2: (128, 128, 3, 3), float32
Block 128x128 loaded. Conv1: (64, 128, 3, 3), Conv2: (64, 64, 3, 3), float32
Block 256x256 loaded. Conv1: (32, 64, 3, 3), Conv2: (32, 32, 3, 3), float32
Block 512x512 loaded. Conv1: (16, 32, 3, 3), Conv2: (16, 16, 3, 3), float32
Block 1024x1024 loaded. Conv1: (8, 16, 3, 3), Conv2: (8, 8, 3, 3), float32
